<a href="https://colab.research.google.com/github/ajay-sainy/notebooks/blob/main/audio_cloning_data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydub
!pip install --upgrade yt-dlp
!pip install tqdm

from tqdm import tqdm

In [ ]:
speaker = "Jordan_2"

#@title Youtube Downloader - Simple Mode { vertical-output: true }
subType = "srv1"
subName = "sub.en." + subType
audioPath = "audio.mp3"
url = "https://www.youtube.com/watch?v=Nb5cBkbQpGY" #@param {type:"string"}
!yt-dlp --write-auto-subs --sub-format {subType} --skip-download {url} -o "subtitle:sub"
#!yt-dlp --list-subs $url
!yt-dlp -f 'ba' -x --audio-format mp3 {url} -o 'audio.%(ext)s'

In [ ]:
from google.colab import drive
drive_path = '/content/drive'
drive.mount(drive_path)

drive_output_folder = drive_path + "/My Drive/audio_clone"
speaker_output_folder = f"{drive_output_folder}/{speaker}"
audio_output_folder = f"{speaker_output_folder}/wavs"

In [ ]:
import xml.etree.ElementTree as ET
import html
import os

tree = ET.parse(subName)
root = tree.getroot()

# &amp;amp; -> & -> and
# &amp;#39; -> you&amp;#39;ll -> you'll -> you will
result = []
for line in root.iter('text'):
    # TODO: skip if contains music etc []
    text = html.unescape(line.text).strip() # TODO: Take care of vocab/Dictonary. Maybe remove/replace "&" and "'"
    start = float(line.attrib["start"])
    #print(start, " -> ", text)
    result.append((start, text))

finalResult = [] # start, end, length, text
for i in range(len(result) - 1):
  # print(item)

  item = result[i]
  nextItem = result[i + 1]

  start = item[0]
  text = item[1]
  length = round(nextItem[0] - start, 3)
  end = nextItem[0]
  if "[" in text:
    print("Skipping", text, item)
  else:
    finalResult.append((start, end, length, text))
print(finalResult[0:5])

In [ ]:
# Frequency analysis
def freq_analysis(arr):
  my_dict = {}
  for item in arr:
    val = int(item[2])
    if val in my_dict:
      my_dict[val] += 1
    else:
      my_dict[val] = 1
  for key in sorted(my_dict):
    print(key, ":", my_dict[key])

In [ ]:
import random

mergedResult = []
choiceArr = [0, 1, 2, 3]

# finalResult
# inputArr = [
#     (0, 1, 1, "1"),
#     (1, 5, 4, "2"),
#     (5, 10, 5, "3"),
#     (10, 11, 1, "4"),
#     (11, 12, 1, "5")
# ]
inputArr = finalResult

i = 0 
while i < len(inputArr):
  mergeLength = random.choice(choiceArr)
  # start, end, length, text
  temp = inputArr[i]
  mergeCount = 1
  for j in range(mergeLength):
    nextIndex = i + j + 1
    if nextIndex < len(inputArr):
      nextItem = inputArr[nextIndex]
      newLength = temp[2] + nextItem[2]
      if newLength < 10:
        text = temp[3]  + " " + nextItem[3]
        temp = (temp[0], nextItem[1], newLength, text)
        mergeCount += 1
      else:
        break
  
  i = i + mergeCount
  mergedResult.append(temp)

In [ ]:
from pydub import AudioSegment
audio = AudioSegment.from_mp3(audioPath)

In [ ]:
from pydub.silence import split_on_silence
# Remove silences in the clips
def removeSilences(inputAudio):
  audio_chunks = split_on_silence(inputAudio
                              ,min_silence_len = 100
                              ,silence_thresh = -45
                              ,keep_silence = 50
                          )

  # Putting the file back together
  combined = AudioSegment.empty()
  for chunk in audio_chunks:
      combined += chunk
  # print("Removed silence (sec):", inputAudio.duration_seconds - combined.duration_seconds)
  return combined

def processForTraining(chunk):
  chunk = removeSilences(chunk)
  chunk = chunk.split_to_mono()[0]
  chunk = chunk.set_frame_rate(22050)
  return chunk

def ensure_dir(path):
  os.makedirs(path, exist_ok=True)



In [ ]:
TOTAL_AUDIO_TO_EXTRACT = 300
final_output_folder = audio_output_folder
# create a directory to store the audio chunks
ensure_dir(final_output_folder)
exportFormat = "wav"
chunksInfo = mergedResult
for i, chunkInfo in enumerate(tqdm(chunksInfo[:TOTAL_AUDIO_TO_EXTRACT])):
  # print(chunkInfo)
  start = chunkInfo[0] * 1000 #Works in milliseconds
  end = chunkInfo[1] * 1000
  chunk = audio[start:end]
  chunkName = str(i) + '.' + exportFormat
  chunkPath = final_output_folder + "/" + chunkName
  chunk = processForTraining(chunk)
  chunk.export(chunkPath, format=exportFormat)
# Trim silence - https://stackoverflow.com/questions/29547218/remove-silence-at-the-beginning-and-at-the-end-of-wave-files-with-pydub

In [ ]:
trainFilepath = f"{speaker_output_folder}/{speaker}_train.txt"
valFilePath = f"{speaker_output_folder}/{speaker}_val.txt"

trainContent = []
for i, item in enumerate(mergedResult[:TOTAL_AUDIO_TO_EXTRACT]):
  trainContent.append(f"{i}.wav|{item[3]}")

valContent = trainContent[:int(len(trainContent)/10)]

with open(trainFilepath, 'w') as trainFile:
  trainFile.writelines("\n".join(trainContent))

with open(valFilePath, 'w') as valFile:
  valFile.writelines("\n".join(valContent))

In [ ]:
!cat $trainFile

In [ ]:
# !rm final/ -r
# !rm audio_chunks/ -r
# !rm /content/drive/MyDrive/audio_clone -r